In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import sklearn
import numpy as np
import pandas as pd 
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  OneHotEncoder
import matplotlib.pyplot as plt
import time 

In [2]:
start=time.time()
file=pd.read_csv('D:/桌面/sonar.csv',header=None)

In [3]:
data=file.iloc[:,:40]
target=file.iloc[:,-1]

In [4]:
data=np.array(data,dtype=float)

In [5]:
target=pd.get_dummies(target).iloc[:,0]

In [6]:
x_train,x_test,y_train,y_test=train_test_split(data,target,test_size=0.25,random_state=5)

In [7]:
positive_data=[]
negative_data=[]
positive_num=0
negative_num=0
for (data,label)in zip(x_train,y_train):
    if label ==1:
        positive_data.append(list(data))
        positive_num+=1
    else:
        negative_data.append(list(data))
        negative_num+=1
    

In [8]:
row,col=np.shape(x_train)

In [9]:
positive=positive_num*1.0/row
negative=1-positive

In [10]:
positive_data=np.array(positive_data)
negative_data=np.array(negative_data)

In [11]:
mean_positive=np.mean(positive_data,axis=0)

In [12]:
mean_negative=np.mean(negative_data,axis=0)

In [13]:
positive_deta=positive_data-mean_positive
negative_deta=negative_data-mean_negative

In [14]:
sigma=[]
for deta in positive_deta:
    deta=deta.reshape(1,col)
    ans = deta.T.dot(deta)
    sigma.append(ans)
for deta in negative_deta:
    deta=deta.reshape(1,col)
    ans = deta.T.dot(deta)
    sigma.append(ans)
sigma=np.array(sigma)
sigma=np.mean(sigma,axis=0)

In [15]:
mean_positive=mean_positive.reshape(1,col)
mean_negative=mean_negative.reshape(1,col)

In [16]:
def Gaussian(x,mean,cov):
    dim=np.shape(cov)[0]
    covdet = np.linalg.det(cov + np.eye(dim) * 0.001)
    covinv = np.linalg.inv(cov + np.eye(dim) * 0.001)
    xdiff = (x - mean).reshape((1, dim))
    prob = 1.0 / (np.power(np.power(2 * np.pi, dim) * np.abs(covdet), 0.5)) * \
    np.exp(-0.5 * xdiff.dot(covinv).dot(xdiff.T))[0][0]
    return prob

In [17]:
def predict(x_test,mean_positive,mean_negetive):
    predict_label=[]
    for data in x_test:
        positive_pro=Gaussian(data, mean_positive, sigma)
        negative_pro=Gaussian(data, mean_negetive, sigma)
        if positive_pro>=negative_pro:
            predict_label.append(1)
        else:
            predict_label.append(0)
    return predict_label

In [18]:
test_predict=predict(x_test,mean_positive,mean_negative)

In [19]:
test_predict=np.array(test_predict)
y_test=np.array(y_test)

In [20]:
accuracy=(test_predict==y_test).sum().item()/len(y_test)

In [21]:
print('用时{:.2f}s,准确率为{:.2f}%'.format(time.time()-start,accuracy*100.0))

用时0.21s,准确率为75.00%
